In [1]:
#subdir in results
#result_path = 'results/simcyp298/'
result_path = 'results/pmc_sim_ami_128/'


# load puptator file
#filelist = ['data/Tafamidis97_filtered.pubtator']
#filelist = ['data/SimCyp298_filtered.pubtator']

# pmc
filelist = ['data/pmc_simvastatin_amiodarone_filtered_128.pubtator']
#filelist = ['data/pmc_simvastatin_erythromycin_filtered_108.pubtator']
#filelist = ['data/pmc_sim_cyp_100_filtered.pubtator']

In [2]:
import os
print('remove snorkel db...')
os.remove("snorkel.db") 
print('snorkel db removed')

remove snorkel db...
snorkel db removed


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

In [4]:
from snorkel import SnorkelSession
from snorkel.parser import CorpusParser, Spacy, StanfordCoreNLPServer
from pubtator import PubTatorDocPreprocessor, PubTatorTagProcessor, PubTatorParser
from time import time

parser = "spacy"
num_procs = 1

start_ts = time()


for fp in filelist:
    doc_preprocessor = PubTatorDocPreprocessor(fp, annotations=False, debug=True)
    #arser = Spacy() if parser == "spacy" else StanfordCoreNLPServer()
    parser = PubTatorParser(stop_on_err=False)
    corpus_parser = CorpusParser(parser=parser)
    corpus_parser.apply(doc_preprocessor, parallelism=num_procs, clear=True)
    end_ts = time()

print("\nDONE in {}".format((time() - start_ts)))

Clearing existing...
Running UDF...
DEBUG: processing doc_id: 5951216 ...
DEBUG: processing doc_id: 3643340 ...
DEBUG: processing doc_id: 4872953 ...
DEBUG: processing doc_id: 5045550 ...
DEBUG: processing doc_id: 6299330 ...
DEBUG: processing doc_id: 5791210 ...
DEBUG: processing doc_id: 4849387 ...
DEBUG: processing doc_id: 2963161 ...
DEBUG: processing doc_id: 5332115 ...
DEBUG: processing doc_id: 6402887 ...
DEBUG: processing doc_id: 3722377 ...
DEBUG: processing doc_id: 6044386 ...
DEBUG: processing doc_id: 3929149 ...
DEBUG: processing doc_id: 6268819 ...
DEBUG: processing doc_id: 2941787 ...
DEBUG: processing doc_id: 5588350 ...
DEBUG: processing doc_id: 4243320 ...
DEBUG: processing doc_id: 4800352 ...
DEBUG: processing doc_id: 6354302 ...
DEBUG: processing doc_id: 4726750 ...
DEBUG: processing doc_id: 4894065 ...
DEBUG: processing doc_id: 5021965 ...
DEBUG: processing doc_id: 6082457 ...
DEBUG: processing doc_id: 6390564 ...
DEBUG: processing doc_id: 2672446 ...
DEBUG: process

In [5]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

Documents: 127
Sentences: 31662


In [6]:
import os


try:  
    os.mkdir(result_path)
except OSError:  
    print ("Creation of the directory {} failed because it may exists".format(result_path))
else:  
    print ("Successfully created the directory %s " % result_path)


Successfully created the directory results/pmc_sim_ami_128/ 


In [7]:
from snorkel.models import Document

all_docs = session.query(Document).all()

print('Amount of docs: {}'.format(len(all_docs)))
with open(result_path + 'doc_mapping.tsv', 'w') as f:
    f.write('{}\t{}'.format('snorkel_id', 'pmid'))
    for doc in all_docs:
        f.write('\n{}\t{}'.format(doc.id, doc.name))
print('Finished')

Amount of docs: 127
Finished


In [8]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

print("Loading all sentences from db...")
all_sents = session.query(Sentence).all()
print("Loading complete!")


print('Amount of sentences: {}'.format(len(all_sents)))

Documents: 127
Sentences: 31662
Loading all sentences from db...
Loading complete!
Amount of sentences: 31662


In [9]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])
candidate_extractor = PretaggedCandidateExtractor(ChemicalDisease, ['Chemical', 'Disease'])

for k, sents in enumerate([all_sents]):
    candidate_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(ChemicalDisease).filter(ChemicalDisease.split == k).count())
    

from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('chemical_disease.lstm')

print("Loading all candidates from db...")
all_cands = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).order_by(ChemicalDisease.id).all()
print("{} candidates load from db!".format(len(all_cands)))


print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id', 'chemical_cid', 'chemical_span', 'disease_cid', 'disease_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "chemical_disease_association.tsv", session, all_cands, all_sents, header_str, 'chemical_cid', 'disease_cid')

  0%|          | 52/31662 [00:00<01:01, 517.35it/s]

Clearing existing...
Running UDF...


100%|██████████| 31662/31662 [00:40<00:00, 779.82it/s] 


Number of candidates: 15005
[LSTM] Loaded model <chemical_disease.lstm>
Loading all candidates from db...
15005 candidates load from db!
Applying LSTM to candidates...


/home/kroll/.conda/envs/snorkel/lib/python3.6/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Saved 15005 marginals
CPU times: user 1min 17s, sys: 7.8 s, total: 1min 25s
Wall time: 38.9 s
LSTM applied!
Storing candidate labels into result file: results/pmc_sim_ami_128/chemical_disease_association.tsv
Amount of candidates: 15005
Load mariginals from db...
Marginals loaded!
Building sentence to document map...
Map built!
Saved 6199 positive predicitions for binary relation!
CPU times: user 47.8 s, sys: 1.35 s, total: 49.2 s
Wall time: 49.2 s


In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

ChemicalGeneInteraction = candidate_subclass('ChemicalGeneInteraction', ['chemical', 'gene'])
candidate_extractor = PretaggedCandidateExtractor(ChemicalGeneInteraction, ['Chemical', 'Gene'])

for k, sents in enumerate([all_sents]):
    candidate_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(ChemicalGeneInteraction).filter(ChemicalGeneInteraction.split == k).count())
    
from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('chemical_gene_interaction.lstm')


print("Loading all candidates from db...")
all_cands = session.query(ChemicalGeneInteraction).filter(ChemicalGeneInteraction.split == 0).order_by(ChemicalGeneInteraction.id).all()
print("{} candidates load from db!".format(len(all_cands)))

print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id','chemical_cid', 'chemical_span', 'gene_cid', 'gene_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "chemical_gene_interaction.tsv", session, all_cands, all_sents, header_str, 'chemical_cid', 'gene_cid')

Clearing existing...


  0%|          | 0/31662 [00:00<?, ?it/s]

Running UDF...


100%|██████████| 31662/31662 [00:24<00:00, 1318.54it/s]


Number of candidates: 9869
[LSTM] Loaded model <chemical_gene_interaction.lstm>
Loading all candidates from db...
9869 candidates load from db!
Applying LSTM to candidates...
Saved 9869 marginals
CPU times: user 52.4 s, sys: 2.72 s, total: 55.2 s
Wall time: 25 s
LSTM applied!
Storing candidate labels into result file: results/pmc_sim_ami_128/chemical_gene_interaction.tsv
Amount of candidates: 9869
Load mariginals from db...
Marginals loaded!
Building sentence to document map...
Map built!
Saved 1847 positive predicitions for binary relation!
CPU times: user 9.35 s, sys: 304 ms, total: 9.65 s
Wall time: 9.65 s


In [12]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

GeneDiseaseInteraction = candidate_subclass('GeneDiseaseInteraction', ['gene', 'disease'])
candidate_extractor = PretaggedCandidateExtractor(GeneDiseaseInteraction, ['Gene', 'Disease'])

for k, sents in enumerate([all_sents]):
    candidate_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(GeneDiseaseInteraction).filter(GeneDiseaseInteraction.split == k).count())
    
from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('gene_disease_interaction.lstm')

print("Loading all candidates from db...")
all_cands = session.query(GeneDiseaseInteraction).filter(GeneDiseaseInteraction.split == 0).order_by(GeneDiseaseInteraction.id).all()
print("{} candidates load from db!".format(len(all_cands)))

print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id','gene_cid', 'gene_span', 'disease_cid', 'disease_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "gene_disease_interaction.tsv", session, all_cands, all_sents, header_str, 'gene_cid', 'disease_cid')

Clearing existing...


  0%|          | 0/31662 [00:00<?, ?it/s]

Running UDF...


100%|██████████| 31662/31662 [00:11<00:00, 2831.14it/s]


Number of candidates: 2318
[LSTM] Loaded model <gene_disease_interaction.lstm>
Loading all candidates from db...
2318 candidates load from db!
Applying LSTM to candidates...
Saved 2318 marginals
CPU times: user 18.9 s, sys: 1.28 s, total: 20.2 s
Wall time: 6.86 s
LSTM applied!
Storing candidate labels into result file: results/pmc_sim_ami_128/gene_disease_interaction.tsv
Amount of candidates: 2318
Load mariginals from db...
Marginals loaded!
Building sentence to document map...
Map built!
Saved 2015 positive predicitions for binary relation!
CPU times: user 5.93 s, sys: 172 ms, total: 6.1 s
Wall time: 5.01 s


In [14]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor


GeneChemicalMetabolism = candidate_subclass('GeneChemicalMetabolism', ['gene', 'chemical'])
candidate_gene_chemical_metabolism_extractor = PretaggedCandidateExtractor(GeneChemicalMetabolism, ['Gene', 'Chemical'])

for k, sents in enumerate([all_sents]):
    candidate_gene_chemical_metabolism_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(GeneChemicalMetabolism).filter(GeneChemicalMetabolism.split == k).count())
    
    
from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('gene_chemical_metabolism.lstm')

print("Loading all candidates from db...")
all_cands = session.query(GeneChemicalMetabolism).filter(GeneChemicalMetabolism.split == 0).order_by(GeneChemicalMetabolism.id).all()
print("{} candidates load from db!".format(len(all_cands)))

print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id','gene_cid', 'gene_span', 'chemical_cid', 'chemical_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "gene_chemical_metabolism.tsv", session, all_cands, all_sents, header_str, 'gene_cid', 'chemical_cid')

  0%|          | 0/31662 [00:00<?, ?it/s]

Clearing existing...
Running UDF...


100%|██████████| 31662/31662 [00:21<00:00, 1463.74it/s]


Number of candidates: 9869
[LSTM] Loaded model <gene_chemical_metabolism.lstm>
Loading all candidates from db...
9869 candidates load from db!
Applying LSTM to candidates...
Saved 9869 marginals
CPU times: user 41.6 s, sys: 1.87 s, total: 43.5 s
Wall time: 25.1 s
LSTM applied!
Storing candidate labels into result file: results/pmc_sim_ami_128/gene_chemical_metabolism.tsv
Amount of candidates: 9869
Load mariginals from db...
Marginals loaded!
Building sentence to document map...
Map built!
Saved 758 positive predicitions for binary relation!
CPU times: user 7.96 s, sys: 252 ms, total: 8.22 s
Wall time: 8.21 s


In [16]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession

session = SnorkelSession()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor

from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor
from snorkel.models import Candidate, candidate_subclass
from snorkel.candidates import PretaggedCandidateExtractor



ChemicalGeneInhibition = candidate_subclass('ChemicalGeneInhibition', ['chemical', 'gene'])
candidate_gene_chemical_inhibit_extractor = PretaggedCandidateExtractor(ChemicalGeneInhibition, ['Chemical', 'Gene'])


for k, sents in enumerate([all_sents]):
    candidate_gene_chemical_inhibit_extractor.apply(sents, split=k, clear=True)
    print("Number of candidates:", session.query(ChemicalGeneInhibition).filter(ChemicalGeneInhibition.split == k).count())
    
from snorkel.learning.pytorch import LSTM

lstm = LSTM(n_threads=10)
lstm.load('chemical_gene_inhibition.lstm')

print("Loading all candidates from db...")
all_cands = session.query(ChemicalGeneInhibition).filter(ChemicalGeneInhibition.split == 0).order_by(ChemicalGeneInhibition.id).all()
print("{} candidates load from db!".format(len(all_cands)))

print("Applying LSTM to candidates...")
%time lstm.save_marginals(session, all_cands)
print("LSTM applied!")

from ksnorkel import KSUtils

header_str = '{}\t{}\t{}\t{}\t{}\t{}\t{}'.format('document_id', 'sentence_id', 'cand_id','chemical_cid', 'chemical_span', 'gene_cid', 'gene_span')
%time KSUtils.save_binary_relation_as_tsv(result_path + "chemical_gene_inhibition.tsv", session, all_cands, all_sents, header_str, 'chemical_cid', 'gene_cid')

Clearing existing...


  0%|          | 0/31662 [00:00<?, ?it/s]

Running UDF...


100%|██████████| 31662/31662 [00:21<00:00, 1459.02it/s]


Number of candidates: 9869
[LSTM] Loaded model <chemical_gene_inhibition.lstm>
Loading all candidates from db...
9869 candidates load from db!
Applying LSTM to candidates...
Saved 9869 marginals
CPU times: user 43.3 s, sys: 2.56 s, total: 45.9 s
Wall time: 25 s
LSTM applied!
Storing candidate labels into result file: results/pmc_sim_ami_128/chemical_gene_inhibition.tsv
Amount of candidates: 9869
Load mariginals from db...
Marginals loaded!
Building sentence to document map...
Map built!
Saved 1059 positive predicitions for binary relation!
CPU times: user 7.95 s, sys: 208 ms, total: 8.16 s
Wall time: 8.15 s
